# Linear Modelling

In this notebook we'll go and start using `ML.NET`. `ML.NET` offers a vast array of high-end modelling tools. The library is built for performance and deployability in mind and there a lot of great choices for linear models like regressions using stochastic gradient descent for training the regression parameters. The "standard" linear regression we know and love from Excel or R is located in the `Microsoft.ML.Mkl.Components` where you can find it as `OLS` (ordinary least squares).

In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: Microsoft.ML, 1.5.5"
#r "nuget: Microsoft.ML.Mkl.Components, 1.5.5"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

As usual we load the data and quickly skim the first couple of rows as a quick sanity check.

In [ ]:
open Deedle
open Plotly.NET

let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

With a univariate time series we wouldn't really have any predictors. We just have the values at their respective points in time $t$. If we want to use this dataset we'll have to do some feature engineering (the bread and butter of the working data scientist). We will do the following things:

- Create a lag of $t_{1}$
- Encode a factor for the day of the week (for the weekly seasonality)
- Encode a factor for the month of the year (for the yearly seasonality)
- Encode a factor for the peak/off-peak behavior (a concept from energy trading)

The last step in preparation before we can create our model is to separate the data into a training set (2015-2018) and a testing set (2019). I won't include 2020 for obvious reasons (the data is super interesting but the impact of a pandemic would warrant its own talk).

In [ ]:
let shiftedValues =
    data?Value
    |> Series.shift -1

let baseDateSeries =
    Seq.zip data.RowKeys data.RowKeys
    |> Series.ofObservations

let dayOfWeek =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.DayOfWeek)

let month =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.Month)

let peakOffPeak =
    baseDateSeries
    |> Series.mapValues (fun dt -> if dt.Hour < 8 || dt.Hour > 19 then "OffPeak" else "Peak")

let dataWithFeatures =
    data
    |> Frame.addCol "Target" shiftedValues
    |> Frame.addCol "DayOfWeek" dayOfWeek
    |> Frame.addCol "Month" month
    |> Frame.addCol "PeakOffPeak" peakOffPeak
    |> Frame.filterRows (fun key _ -> key.Year < 2020)
    |> Frame.dropSparseRows

let dataTrain =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year < 2019)

let dataTest =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year >= 2019)

dataTrain
|> Frame.skip 3

It makes also sense to look at the end of our training set to know exactly what comes before our evaluation data.

In [ ]:
dataTrain |> Frame.takeLast 3

We could work with all series in isolation and combine them to a new data view in ML.NET. `Deedle` offers a typed interface which does a lot of the ground work for us so I'd rather go with that.

In [ ]:
type ILoadRow =
    abstract member Ticks: float32 with get
    abstract member Value: float32 with get
    abstract member Target: float32 with get
    abstract member DayOfWeek: string with get
    abstract member Month: string with get
    abstract member PeakOffPeak: string with get

let (trainKeys: DateTime seq, trainRows: ILoadRow seq) =
    dataTrain.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

let testKeys, testRows =
    dataTest.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

Seq.length trainRows, Seq.length testRows

In [ ]:
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Trainers
open Microsoft.ML.Transforms
open FSharp.Stats.Correlation

F# makes working with types really easy so I'd define types and mappings for all different pipeline steps in isolation of each other.

In [ ]:
[<CLIMutable>]
type ForecastInput =
    { Ticks: float32
      Value: float32
      [<ColumnName("Label")>]Target: float32
      DayOfWeek: string
      Month: string
      PeakOffPeak: string }

    static member FromILoadRows (row: ILoadRow) =
        { Ticks = row.Ticks
          Value = row.Value
          Target = row.Target
          DayOfWeek = row.DayOfWeek
          Month = row.Month
          PeakOffPeak = row.PeakOffPeak }

[<CLIMutable>]
type ForecastResult =
    { [<ColumnName("Score")>]LoadForecast: float32 }

The `ML.NET` modelling process includes two parts:

- Transformations to make the data workable for a linear regression
- The trainer which gets fitted

We only work with numerical values so first we have to one hot encode all nominal variables. After that we can concatenate the dummies as well as the running index component (which hopefully captures the trend) inthe model's feature vector.

In [ ]:
let downCastPipeline (pipeline: IEstimator<'a>) =
    match pipeline with
    | :? IEstimator<ITransformer> as p -> p
    | _ -> failwith $"The pipeline has to be an instance of IEstimator<ITransformer> but was %A{pipeline.GetType()}"

let mlContext = MLContext(seed = 42)
let defInp = Unchecked.defaultof<ForecastInput>
let dayOneHot = "DayOfWeekOneHot"
let monthOneHot = "MonthOneHot"
let peakOneHot = "PeakOffPeakOneHot"

let processingPipeline =
    EstimatorChain()
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(dayOneHot, nameof defInp.DayOfWeek))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(monthOneHot, nameof defInp.Month))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(peakOneHot, nameof defInp.PeakOffPeak))
        .Append(mlContext.Transforms.Concatenate("Features", [| dayOneHot; monthOneHot; peakOneHot; nameof defInp.Ticks |]))
    |> downCastPipeline

let trainerOptions = OlsTrainer.Options(CalculateStatistics = true)
let trainer =
    mlContext.Regression.Trainers.Ols(trainerOptions)
    |> downCastPipeline

With all this preparation done we can finally fit and evaluate our model.

In [ ]:
let dataViewTrain = mlContext.Data.LoadFromEnumerable<ForecastInput>(trainRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))
let dataViewTest = mlContext.Data.LoadFromEnumerable<ForecastInput>(testRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))

let trainingPipeline = processingPipeline.Append(trainer)
let trainedModel = trainingPipeline.Fit(dataViewTrain)

let transformedData = trainedModel.Transform(dataViewTest)
let predictions = mlContext.Data.CreateEnumerable<ForecastResult>(transformedData, reuseRowObject = false) |> Seq.toList

mlContext.Regression.Evaluate(transformedData)

A $R^2$ of roughly 59% isn't exactly good but it isn't too shabby either for such a naive model. Naturally those values are hard to interpret without actually looking at the fitted values in relation to the true data.

In [ ]:
open Plotly.NET

let predVals = predictions |> Seq.map (fun p -> p.LoadForecast)
let actualVals = testRows |> Seq.map (fun r -> r.Target)

let predChart =
    Seq.zip testKeys predVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip testKeys actualVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

Most trainers in `ML.NET` give you extra information if you ask them nicely. The OLS trainer contains a couple of statistics. Unfortunately p-Values arent reliable at the moment. See [this issue](https://github.com/dotnet/machinelearning/issues/5696) for more context. This isn't too much of a problem really as linear regressions are very portable. We could also just use `FSharp.Stats` or R to compute the regression based on the same data and get addtional insights into the true p-Values and t-Statistics.

In [ ]:
let model = (trainedModel.LastTransformer :?> RegressionPredictionTransformer<Microsoft.ML.Trainers.OlsModelParameters>).Model
model

Another great way to diagnose the model is to look at the actual values versus the forecasted values on a scatterplot. If the model were perfect all values would lie on a 45° diagonal. This is never the case but the way in which it differs from the optimum can tell us a lot. We can also look at the correlation between the actual and predicted values.

In [ ]:
let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

If we are happy with our model we can save it to disk for later use.

In [ ]:
let modelDirectory = "../models"
let linearModel = modelDirectory + "/linear_model.zip"

mlContext.Model.Save(trainedModel, dataViewTrain.Schema, linearModel)